In [1]:
using Pkg
Pkg.activate("..")
Pkg.add("BenchmarkTools")
using Test
using BenchmarkTools

  Activating new project at `c:\Users\wenbl13\Desktop\Coding-Club\coding club`


    Updating registry at `C:\Users\wenbl13\.julia\registries\General.toml`


   Resolving package versions...


    Updating `C:\Users\wenbl13\Desktop\Coding-Club\coding club\Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
    Updating `C:\Users\wenbl13\Desktop\Coding-Club\coding club\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [682c06a0] + JSON v0.21.3
  [69de0a69] + Parsers v2.5.8
  [21216c6a] + Preferences v1.3.0
  [66db9d55] + SnoopPrecompile v1.0.3
  [56f22d72] + Artifacts
  [ade2ca70] + Dates
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [a63ad114] + Mmap
  [de0858da] + Printf
  [9abbd945] + Profile
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
  [fa267f1f] + TOML v1.0.0
  [cf7118a7] + UUIDs
  [4ec0a83e] + Unicode
  [e66e0078] + CompilerSupportLibraries_jll v1.0.1+0
  [4536629a] + OpenBLAS_jll v0.3.20+0
  [8e850b90] + libblastrampoline_jll v5.1.1+0


# 0. Utils

In [2]:
# This function in-place swaps i-th el and j-th el in arr.
function swap!(arr, i, j)
    temp = arr[i]
    arr[i] = arr[j]
    arr[j] = temp
end

swap! (generic function with 1 method)

# 1. Double loop(Bubble): time = $O(n^2)$, space = $O(n)$
in-place space complexity = $O(1)$

In [3]:
function sort_Double_loop!(arr)
    l = length(arr)
    for i = 1:l-1
        for j = i+1:l
            if arr[i] > arr[j]
                swap!(arr, i, j)
            end
        end
    end
    return arr
end

function sort_Double_loop(arr_org)
    arr = copy(arr_org)
    l = length(arr)
    for i = 1:l-1
        for j = i+1:l
            if arr[i] > arr[j]
                swap!(arr, i, j)
            end
        end
    end
    return arr
end

sort_Double_loop (generic function with 1 method)

Test correctness

In [4]:
for i = 1:500 # 1000 sizes
    for j = 1:100 # 500 random cases for size = i
        test_case = round.(Int, rand(i) .* 100)
        @test sort_Double_loop(test_case)==sort(test_case)
    end
end

# 2. Merge sort: time = $O(n\times logn)$, space = $O(n\times logn) \rightarrow O(n)$
Cannot achieve as in-place.

In [5]:
function sort_Merge(arr_org)
    arr = copy(arr_org)
    return sort_Merge_rec(arr)
end

function sort_Merge2(arr_org)
    arr = copy(arr_org)
    arr_copy = similar(arr)
    return sort_Merge_rec(arr, arr_copy)
end

function sort_Merge_rec(arr)
    l = length(arr)
    # base case
    if l == 1
        return arr
    end
    # rec
    mid = floor(Int, l/2)
    L_rslt = sort_Merge_rec(arr[1 : mid]) # sort left section, length = mid
    R_rslt = sort_Merge_rec(arr[mid+1 : l]) # sort right section, length = l - mid

    pointer_left, pointer_right, pointer_rslt = 1, 1, 1
    while pointer_left<=mid && pointer_right<=l-mid
        if L_rslt[pointer_left]<R_rslt[pointer_right]
            arr[pointer_rslt] = L_rslt[pointer_left]
            pointer_left+=1
        else
            arr[pointer_rslt] = R_rslt[pointer_right]
            pointer_right+=1
        end
        pointer_rslt += 1
    end
    # if anything left in L_rslt
    while pointer_left<=mid
        arr[pointer_rslt] = L_rslt[pointer_left]
        pointer_left += 1
        pointer_rslt += 1
    end
    # if anything left in R_rslt
    while pointer_right<=l-mid
        arr[pointer_rslt] = R_rslt[pointer_right]
        pointer_right += 1
        pointer_rslt += 1
    end
    return arr
end

function sort_Merge_rec(arr, arr_copy)
    l = length(arr)
    # base case
    if l == 1
        return arr
    end
    # rec
    mid = floor(Int, l/2)
    L_rslt = sort_Merge_rec(@views(arr[1 : mid]), @views(arr_copy[1 : mid])) # sort left section, length = mid
    R_rslt = sort_Merge_rec(@views(arr[mid+1 : l]), @views(arr_copy[mid+1 : l])) # sort right section, length = l - mid

    pointer_left, pointer_right, pointer_rslt = 1, 1, 1
    while pointer_left<=mid && pointer_right<=l-mid
        if L_rslt[pointer_left]<R_rslt[pointer_right]
            arr_copy[pointer_rslt] = L_rslt[pointer_left]
            pointer_left+=1
        else
            arr_copy[pointer_rslt] = R_rslt[pointer_right]
            pointer_right+=1
        end
        pointer_rslt += 1
    end
    # if anything left in L_rslt
    while pointer_left<=mid
        arr_copy[pointer_rslt] = L_rslt[pointer_left]
        pointer_left+=1
        pointer_rslt += 1
    end
    # if anything left in R_rslt
    while pointer_right<=l-mid
        arr_copy[pointer_rslt] = R_rslt[pointer_right]
        pointer_right+=1
        pointer_rslt += 1
    end
    arr[:] = arr_copy
    return arr
end

sort_Merge_rec (generic function with 2 methods)

Test correctness

In [6]:
for i = 1:500 # 1000 sizes
    for j = 1:100 # 500 random cases for size = i
        test_case = round.(Int, rand(i) .* 100)
        @test sort(test_case) == sort_Merge(test_case)
    end
end

In [7]:
for i = 1:500 # 1000 sizes
    for j = 1:100 # 500 random cases for size = i
        test_case = round.(Int, rand(i) .* 100)
        @test sort(test_case) == sort_Merge2(test_case)
    end
end

# 3. Quick sort: time = $O(n^2)$(worst-case), $O(nlogn)$(amortized) , space = O(n)
in-place space complexity = $O(1)$

In [8]:
function sort_quick(arr_org)
    arr = copy(arr_org)
    sort_quick_rec(arr, 1, length(arr))
    return arr
end

function sort_quick2(arr_org)
    arr = copy(arr_org)
    # println(arr)
    sort_quick_rec_random_pivot(arr, 1, length(arr))
    return arr
end

# aaaaaaaaaxxxxxxxxxxxbbbbbbbb
#          i         j

function sort_quick_rec(arr, l, r)
    # base case
    if l >= r
        return
    end
    # rec
    pivot = arr[l]
    i, j = l, r
    while i<=j
        if arr[i] <= pivot
            i+=1
        else
            swap!(arr,i,j)
            j-=1
        end
    end
    swap!(arr,l,i-1)
    sort_quick_rec(arr, l, i-2)
    sort_quick_rec(arr, i, r)
end


# 0 <= rand < 1
# 0 <= rand * (right-left+1) < right-left+1
# left <= rand * (right-left+1) + left < right+1
# left <= floor(rand * (right-left+1) + left) <= right

# Random pivot
function sort_quick_rec_random_pivot(arr, l, r)
    # println("[$l --> $r]:")
    # base case
    if l >= r
        return
    end
    # rec
    rand_idx = rand(l:r)
    pivot = arr[rand_idx]
    i, j = l, r
    while i<=j
        if arr[i] >= pivot
            i+=1
        else
            swap!(arr,i,j)
            # trace index of pivot
            if rand_idx == i 
                rand_idx = j
            end
            if rand_idx == j 
                rand_idx = i
            end
            j-=1
        end
    end
    swap!(arr,rand_idx,i-1)
    sort_quick_rec_random_pivot(arr, l, i-2)
    # avoided i-1 th
    sort_quick_rec_random_pivot(arr, i, r)
end

sort_quick_rec_random_pivot (generic function with 1 method)

Test Correctness

In [9]:
for i = 1:500 # 1000 sizes
    for j = 1:100 # 500 random cases for size = i
        test_case = round.(Int, rand(i) .* 100)
        @test sort(test_case) == sort_quick(test_case)
    end
end

In [10]:
temp = 0
for i = 1:500 # 1000 sizes
    for j = 1:100 # 500 random cases for size = i
        test_case = round.(Int, rand(i) .* 100)
        @test sort(test_case) == sort_quick2(test_case)
    end
end

# 4. Counting sort: time = $O(n+K)$, space = $O(n+K)$
where $K = max(arr) - min(arr) + 1$

In [11]:
function sort_Counting(arr)
    rslt = similar(arr)
    arr_min, arr_max = minimum(arr), maximum(arr)
    counters_l = arr_max-arr_min+1
    counters = zeros(Int, counters_l)
    # first pass
    for v in arr
        counters[v-arr_min+1] += 1
    end
    # second pass
    for i = 2:counters_l
        counters[i] += counters[i-1]
    end
    # third pass
    for v in arr
        rslt[counters[v-arr_min+1]] = v
        counters[v-arr_min+1] -= 1
    end
    return rslt
end

sort_Counting (generic function with 1 method)

Test Correctness

In [12]:
for i = 1:500 # 1000 sizes
    for j = 1:100 # 500 random cases for size = i
        test_case = round.(Int, rand(i) .* 100)
        @test sort(test_case) == sort_Counting(test_case)
    end
end

# Benchmark

In [13]:
input_mini = round.(Int, rand(10) .* 6)
input_small = round.(Int, rand(100) .* 10000)
input_large = round.(Int, rand(10000) .* 10000)
input_large_with_bigger_range = round.(Int, rand(10000) .* 100000)
input_worst_case = collect(1:10000);

## Random inputs

In [26]:
@benchmark sort_Double_loop!($input_large)

BenchmarkTools.Trial: 224 samples with 1 evaluation.
 Range (min … max):  21.928 ms …  22.731 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.308 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   22.325 ms ± 150.445 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                     ▅ ▂  ▁▅▃█▃▁▂ ▃▁     ▁▃                     
  ▃▃▁▃▁▃▄▄▃▁▄▄▃▁▃▆▃▅▆█▇█▇▇███████▆███▅█▇███▆▇▄▄▁▄▅▆▄▃▃▃▄▁▅▁▃▁▃ ▄
  21.9 ms         Histogram: frequency by time         22.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
@benchmark sort_Double_loop($input_large)

BenchmarkTools.Trial: 47 samples with 1 evaluation.
 Range (min … max):  107.090 ms … 112.120 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     107.399 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   107.652 ms ± 827.872 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▆▃█▃                                                          
  █████▁▇▇▆▁▄▁▄▄▁▁▁▄▁▁▄▁▁▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  107 ms           Histogram: frequency by time          112 ms <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [15]:
@benchmark sort_Merge($input_large)

BenchmarkTools.Trial: 4483 samples with 1 evaluation.
 Range (min … max):  997.583 μs …   3.472 ms  ┊ GC (min … max): 0.00% … 62.64%
 Time  (median):       1.012 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.114 ms ± 438.104 μs  ┊ GC (mean ± σ):  8.93% ± 14.32%

  █▃                                                         ▁▁  
  ██▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▇██ █
  998 μs        Histogram: log(frequency) by time       3.03 ms <

 Memory estimate: 2.22 MiB, allocs estimate: 20006.

In [16]:
@benchmark sort_Merge2($input_large)

BenchmarkTools.Trial: 5484 samples with 1 evaluation.
 Range (min … max):  885.708 μs …  2.816 ms  ┊ GC (min … max): 0.00% … 67.12%
 Time  (median):     903.209 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   910.616 μs ± 85.489 μs  ┊ GC (mean ± σ):  0.53% ±  3.53%

         ▇█▄ ▁                                                  
  ▁▂▃▂▂▃██████▇▇▇▆▆█▆▇▅▆▇▆▆▅▅▆▅▅▄▅▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁ ▃
  886 μs          Histogram: frequency by time          946 μs <

 Memory estimate: 156.34 KiB, allocs estimate: 4.

In [17]:
@benchmark sort_quick($input_large)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  395.959 μs …  2.461 ms  ┊ GC (min … max): 0.00% … 82.94%
 Time  (median):     403.375 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   408.183 μs ± 64.933 μs  ┊ GC (mean ± σ):  0.63% ±  3.21%

      ▁▄▃▃█▇▅                                                   
  ▂▂▂▄██████████▆▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  396 μs          Histogram: frequency by time          442 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [18]:
@benchmark sort_quick2($input_large)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  435.209 μs …  2.660 ms  ┊ GC (min … max): 0.00% … 81.55%
 Time  (median):     471.084 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   475.352 μs ± 68.463 μs  ┊ GC (mean ± σ):  0.58% ±  3.16%

                   ▃▂▅▅▇███▇▇▅▅▃▁                               
  ▂▁▂▁▂▂▂▂▂▃▃▃▄▅▆▇████████████████▆▅▅▅▄▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂ ▄
  435 μs          Histogram: frequency by time          524 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [19]:
@benchmark sort_Counting($input_large)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  25.791 μs …  1.989 ms  ┊ GC (min … max):  0.00% … 97.85%
 Time  (median):     32.750 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   36.545 μs ± 64.366 μs  ┊ GC (mean ± σ):  10.01% ±  5.60%

                  ▁▇█▅▇▅▂                                      
  ▂▂▃▄▄▃▂▂▁▂▂▂▂▂▃▅███████▇▅▅▆▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  25.8 μs         Histogram: frequency by time        46.3 μs <

 Memory estimate: 156.34 KiB, allocs estimate: 4.

In [20]:
@benchmark sort($input_large)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  296.542 μs …  1.624 ms  ┊ GC (min … max): 0.00% … 80.42%
 Time  (median):     308.375 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   312.016 μs ± 48.640 μs  ┊ GC (mean ± σ):  0.63% ±  3.21%

           ▃▃  ▂█▅ ▁▂▁                                          
  ▂▂▁▁▃▃▃▂▆███▅███▇████▆▅▅▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  297 μs          Histogram: frequency by time          342 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

## Random input, but with wider range

In [21]:
@benchmark sort_Counting($input_large_with_bigger_range)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   77.541 μs …   1.411 ms  ┊ GC (min … max):  0.00% … 87.38%
 Time  (median):     117.625 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   131.530 μs ± 108.692 μs  ┊ GC (mean ± σ):  10.89% ± 11.56%

    ▂█                                                           
  ▃▃██▃▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▂
  77.5 μs          Histogram: frequency by time          850 μs <

 Memory estimate: 859.34 KiB, allocs estimate: 4.

In [22]:
@benchmark sort($input_large_with_bigger_range)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  289.167 μs …  1.508 ms  ┊ GC (min … max): 0.00% … 77.56%
 Time  (median):     299.083 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   303.048 μs ± 40.976 μs  ┊ GC (mean ± σ):  0.53% ±  3.08%

        ▁▃▆▄▅█▆▇▄▃▁                                             
  ▁▁▄▃▇▅████████████▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▃▄▃▂▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  289 μs          Histogram: frequency by time          335 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

## Worst case for quick sort

In [23]:
@benchmark sort_quick($input_worst_case)

BenchmarkTools.Trial: 5594 samples with 1 evaluation.
 Range (min … max):  872.917 μs …  3.232 ms  ┊ GC (min … max): 0.00% … 72.08%
 Time  (median):     886.084 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   892.759 μs ± 73.036 μs  ┊ GC (mean ± σ):  0.32% ±  2.63%

       ▁▁▇▆▅█▅▅▃                                                
  ▁▁▃▂▄██████████▅▄▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  873 μs          Histogram: frequency by time          943 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [24]:
@benchmark sort_quick2($input_worst_case)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  233.458 μs …  1.475 ms  ┊ GC (min … max): 0.00% … 81.63%
 Time  (median):     255.042 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   258.656 μs ± 42.353 μs  ┊ GC (mean ± σ):  0.65% ±  3.24%

              ▂▄▅▅▇▆██▇▆▅▅▃▃▁▂ ▁                                
  ▁▁▁▁▁▂▂▃▄▅▇███████████████████▇▆▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁ ▄
  233 μs          Histogram: frequency by time          295 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [25]:
@benchmark sort($input_worst_case)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  47.875 μs …  1.180 ms  ┊ GC (min … max): 0.00% … 94.08%
 Time  (median):     51.375 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.959 μs ± 33.415 μs  ┊ GC (mean ± σ):  2.54% ±  3.82%

    ▃▁          ▃█▄▆█▄▄                                        
  ▂▅██▅▄▃▂▂▂▁▁▂▄████████▅▄▃▃▂▂▂▃▃▆▅▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂ ▃
  47.9 μs         Histogram: frequency by time        59.3 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.